#The Final model - VGG19 and SVM

In [1]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import cv2
from prettytable import PrettyTable
from sklearn.model_selection import train_test_split
import random
from skimage.util import view_as_windows
from matplotlib.pyplot import imread, imsave

from keras.preprocessing.image import ImageDataGenerator

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.callbacks import ModelCheckpoint,LearningRateScheduler, EarlyStopping, TensorBoard,ReduceLROnPlateau
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPooling2D,MaxPooling1D, Concatenate, Flatten, Dropout, Dense, Conv1D,MaxPool2D,BatchNormalization
import tensorflow as tf
import datetime

from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.utils import shuffle
from scipy.sparse import csr_matrix

from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
import keras
from keras.applications import *

from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.vgg19 import preprocess_input
from keras.models import Model, load_model


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


##Initialization

In [119]:
#Initialise all the director paths and fetch the filenames from this directory. 
#The dataset consists of images belonging to Van Gogh and other artists. The images are segregated into separate folders for train and test.
#The train directory and test directory each have two folders - vg and nvg. 
#vg folder has images belonging to Van Gogh
#nvg folder has images belonginh to Non Van Gogh


#directory paths
rootdir = '/content/drive/MyDrive/Colab Notebooks/29. Identification of Van Gogh paintings/vgdb_2016/vgdb_2016'

#Only test data
testdir = rootdir + '/test'
testdir_vg = testdir + '/vg'
testdir_nvg = testdir + '/nvg'

#filenames
test_vg_data = os.listdir(testdir_vg)
test_nvg_data = os.listdir(testdir_nvg)

#Models

#vgg19 model
vgg19model = VGG19(include_top = False, weights = 'imagenet')

#saved SVM model
filename = rootdir+'/final_model_svm'
clf = pickle.load(open(filename, 'rb'))


##Utility Functions

In [11]:
#Fusion methods 

''',
Find the farthest point from the prediction probabilities for each image
'''
def agg_pred_far(pred):

  arr_pos = []
  arr_neg = []
  
  for predItem in pred:
    if(predItem >= 0):
      arr_pos.append(predItem)
    else:
      arr_neg.append(predItem)

  
  max_pos = np.max(arr_pos) if(len(arr_pos) > 0) else 0  
  
  max_neg = np.abs(np.min(arr_neg)) if(len(arr_neg) > 0) else 0

  cl = 1 if(max_pos > max_neg) else 0
  
  return cl

'''
Find the mean from the prediction probabilities for each image
'''
def agg_pred_mean(pred):
  arr_pos = []
  arr_neg = []

  for predItem in pred:
    if(predItem >= 0):
      arr_pos.append(predItem)
    else:
      arr_neg.append(predItem)


  #arr_pos = pred[pred >= 0]
  avg_pos = np.mean(arr_pos) if(len(arr_pos) > 0) else 0
  
  #arr_neg = pred[pred <= 0]
  avg_neg = np.abs(np.mean(arr_neg)) if(len(arr_neg) > 0) else 0

  cl = 1 if(avg_pos > avg_neg) else 0
  
  return cl

'''
Find the median from the prediction probabilities for each image
'''
def agg_pred_median(pred):

  arr_pos = []
  arr_neg = []

  for predItem in pred:
    if(predItem >= 0):
      arr_pos.append(predItem)
    else:
      arr_neg.append(predItem)
  #arr_pos = pred[pred >= 0]
  avg_pos = np.median(arr_pos) if(len(arr_pos) > 0) else 0
  
  #arr_neg = pred[pred <= 0]
  avg_neg = np.abs(np.median(arr_neg)) if(len(arr_neg) > 0) else 0

  cl = 1 if(avg_pos > avg_neg) else 0
  
  return cl


In [101]:

no_of_patches =20

'''This function is used to generate patches for a particular image. 
The number of patches and patch_size are passed as function parameters.
The function generates the number of patches as the given parameter'''
def get_patches_for_img(img_path, patch_size, no_of_patches):
    
    patches = []

    #read the image at the given path
    img = cv2.imread(img_path, 1)    
    image_height = img.shape[0]
    image_width = img.shape[1]

    #Subtract patch_size from image's height and width to avoid out of bounds error
    range_x = image_height - patch_size
    range_y = image_width - patch_size

    #Generate patches for each image. The number of patches are passed as parameter.
    for i in range(no_of_patches):
        
        #Generate patch from random area of the image
        x = np.random.randint(low = 0, high = range_x)
        y = np.random.randint(low = 0, high = range_y)

        #The patch is calculated by adding the patch_size to both x and y co-ordinates
        patch = img[x : x+patch_size, y : y+patch_size, :]
        patches.append(patch)

    return patches

'''This function creates a feature array for a patch. 
It uses the VGG 19 to pre-process and predict the feature array for the patch. '''
def get_patch_feature_vgg19(patch):

    patch_input = np.expand_dims(patch, axis = 0)             #add an extra dimension for batch
    patch_preprocessed_input = preprocess_input(patch_input)               

    p_feature = vgg19model.predict(patch_preprocessed_input)           
    p_feature = p_feature.reshape(25088)
    
    return p_feature


##Prediction Functions

In [118]:
''' This function can be used to obtain the predictions when the filenames, directory paths and actual values are passed.
The number of patches by default are 20 but can be changed.
The predictions are obtained by reading the image from the given directory path, generate random patches ,
use VGG19 to classify the patches and then use SVM to generate probabilities.These probabilities are returned as the 
model's predictions. 
 '''

def get_predictions(filename_list,dirnameArr,no_of_patches=20):
  
  predictions=[]

  for idx,file_name in enumerate(filename_list): 
    patch_pred = []                                 
    
    img_path = dirnameArr[idx] + '/' + file_name
    patches = get_patches_for_img(img_path, patch_size, no_of_patches)
    
    for patch in patches:                                            
        patch_feature = get_patch_feature_vgg19(patch)
        
        pred_proba = clf.decision_function([patch_feature])      
        
        patch_pred.append(pred_proba)  
        

    predictions.append(patch_pred)                                  
    

  return  predictions 


In [115]:
''' This function can be used to obtain the f1 scores when the filenames, directory paths and actual values are passed.
The number of patches by default are 20 but can be changed.
The predictions are obtained by reading the image from the given directory path, generate random patches ,
use VGG19 to classify the patches and then use SVM to generate probabilities. 
The F1-scores are computed over three fusion methods - Median, Mean and Far. The results from these three
methods are returned. 
 '''

def get_metric_f1_score(filename_list,dirnameArr,y_true,no_of_patches=20):
  
  #get the predictions
  predictions = get_predictions(filename_list,dirnameArr,no_of_patches)   

  y_pred_mean= []
  y_pred_median= []
  y_pred_far=[]
  f1_score_median=[]
  f1_score_mean=[]
  f1_score_far = []
  
  #Compute the F1-scores

  for item in predictions:
    y_pred_median.append(agg_pred_median(item))
    y_pred_mean.append(agg_pred_mean(item))
    y_pred_far.append(agg_pred_far(item))
    
  f1_score_median = f1_score(y_true, y_pred_median)
  f1_score_mean = f1_score(y_true, y_pred_mean)
  f1_score_far = f1_score(y_true, y_pred_far)
  
  return f1_score_median,f1_score_mean,f1_score_far

In [116]:
''' This function can be used to obtain the f1 scores when predictions and the actual values are passed.
The F1-scores are computed over three fusion methods - Median, Mean and Far. The results from these three
methods are returned. 
 '''
def get_f1(predictions,y_true):
  
  y_pred_mean= []
  y_pred_median= []
  y_pred_far=[]
  f1_score_median=[]
  f1_score_mean=[]
  f1_score_far = []
  

  for item in predictions:
    y_pred_median.append(agg_pred_median(item))
    y_pred_mean.append(agg_pred_mean(item))
    y_pred_far.append(agg_pred_far(item))
    
  f1_score_median = f1_score(y_true, y_pred_median)
  f1_score_mean = f1_score(y_true, y_pred_mean)
  f1_score_far = f1_score(y_true, y_pred_far)
  
  return f1_score_median,f1_score_mean,f1_score_far

##Data Preparation

In [78]:
test_data = []
test_data.extend(test_vg_data)
test_data.extend(test_nvg_data)
print(len(test_data))

67


In [81]:
y_1 = [1]*25
y_0 = [0]*42

y_true=[]
y_true.extend(y_1)
y_true.extend(y_0)
print(len(y_true))

67


In [98]:
dirpaths = []

vg_path_arr = ['/content/drive/MyDrive/Colab Notebooks/29. Identification of Van Gogh paintings/vgdb_2016/vgdb_2016/test/vg']*25

nvg_path_arr = ['/content/drive/MyDrive/Colab Notebooks/29. Identification of Van Gogh paintings/vgdb_2016/vgdb_2016/test/nvg']*42

dirpaths.extend(vg_path_arr)
dirpaths.extend(nvg_path_arr)

##VGG19 and SVM on Test Data

In [102]:
pred_values = get_predictions(test_data,dirpaths)

In [111]:
f1_median,f1_mean,f1_far,y_med,y_mean,y_far = get_f1(pred_values,y_true)
print("F1 scores with different Fusion methods")
print("="*200)
print("F1 score for test data - Median is ", f1_median)
print("F1 score for test data - Mean is", f1_mean)
print("F1 score for test data - Far is", f1_far)

F1 scores with different Fusion methods
F1 score for test data - Median is  0.9056603773584904
F1 score for test data - Mean is 0.888888888888889
F1 score for test data - Far is 0.8846153846153846


Various models were implemented to find the model which can most accurately predict Van Gogh's paintings. The models that were implemented include, VGG19,ResNet, EfficientNet and their customised versions. The best performing was however VGG19.

All models gave satisfying scores of over 80%

Once again, here VGG19 gives good F1 score of 0.91 for the test data.

With the help of transfer learning with VGG19, we can classify the paintings of VanGogh and other artists,
